#### Notebook to feature engineer fraction of time co was above 90th percentile

In [17]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 15)

In [18]:
PATH_1=os.path.join('..','..','..','..','Weather_Data','CAMS','2007','2007_multi_level_26_variables.nc')
ml_26_variables = xr.open_dataset(PATH_1) ## multilevel 26 variables
ml_26_variables = ml_26_variables.squeeze(dim="model_level") ## remove the model_level dimension
ml_26_variables=ml_26_variables.drop_vars('model_level') ## the dropped dimension becomes a column, so remove that
ml_26_variables['longitude'] = ml_26_variables['longitude']-360 ## necessary because of longitude data in multi-level case
ml_26_variables
propane=ml_26_variables['c3h8'] ## get only carbon monoxide variable
propane

<xarray.DataArray 'c3h8' (valid_time: 2920, latitude: 34, longitude: 78)> Size: 31MB
[7743840 values with dtype=float32]
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 23kB 2007-01-01 ... 2007-12-31T21...
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B -124.8 -124.1 -123.3 ... -67.85 -67.1
Attributes: (12/31)
    GRIB_paramId:                             217047
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      2652
    GRIB_typeOfLevel:                         hybrid
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_pv:                                  [0.00000000e+00 2.00000000e+01 ...
    GRIB_shortName:                           c3h8
    GRIB_units:                               kg kg**-1
    long_name:                                Propane
    units:                                    kg kg**-1
    standard_name:                            mass_fraction_of_propane_in_air

In [19]:
propane.to_dataframe()

c3h8
valid_time          latitude           longitude                             
2007-01-01 00:00:00 49.250000000000000 -124.849999999999994 0.000000001875653
                                       -124.099999999999994 0.000000002563290
                                       -123.349999999999994 0.000000003674196
                                       -122.599999999999994 0.000000003333358
                                       -121.849999999999994 0.000000001574620
...                                                                       ...
2007-12-31 21:00:00 24.500000000000000 -70.100000000000023  0.000000000116427
                                       -69.350000000000023  0.000000000112460
                                       -68.600000000000023  0.000000000111174
                                       -67.850000000000023  0.000000000109982
                                       -67.100000000000023  0.000000000108913

[7743840 rows x 1 columns]

In [20]:
PATH_2=os.path.join('..','CAMS_79_variables_2007.pkl')
get_data = pd.read_pickle(PATH_2) ## load the data with all variables
get_data.head()

,NAME,fips,u10,v10,d2m,t2m,bcaod550,duaod550,lsm,msl,omaod550,pm1,pm2p5,pm10,ssaod550,suaod550,z,sp,aod469,aod550,aod670,aod865,aod1240,tcco,tc_c2h6,tchcho,tc_h2o2,tc_oh,tc_c5h8,tc_ch4,tc_hno3,tcno2,tc_no,gtco3,tc_pan,tc_c3h8,tcso2,tcwv,co,aermr04,aermr05,aermr06,c2h6,hcho,h2o2,aermr09,aermr07,aermr10,aermr08,oh,c5h8,hno3,no2,no,go3,pan,c3h8,aermr01,aermr02,aermr03,q,aermr11,so2,t,lai_hv,lai_lv,asn,sd,tcw,aermssdus,aermssdum,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
1536,Autauga,1001,-0.145081495233009,0.037969070287171,283.659580785162746,292.327994006618042,0.008950339167969,0.008734354045796,0.986464531564209,101847.023645228589885,0.101791173978126,0.000000013802305,0.000000017315602,0.000000024071584,0.009777383694107,0.098779543214663,1368.670051439976305,100190.286124633479631,0.279607349895111,0.228032791750708,0.173623018984247,0.118622767783307,0.066526604474522,0.000935456032256,0.000003565167024,0.000004441778917,0.000009180281274,0.000000004871132,0.000003467643716,0.009846869936512,0.000006204630253,0.000003458695721,0.000000466749385,0.006357170731654,0.000009708567306,0.000000666746841,0.000004086999738,25.884526430341936,0.000000193455430,0.000000000539127,0.000000000924778,0.000000000306597,0.000000000621535,0.000000004189675,0.000000000879831,0.000000000266316,0.000000009475496,0.000000000250366,0.000000004669512,0.000000000000023,0.000000020141195,0.000000001098598,0.000000008048469,0.000000000521161,0.000000063262249,0.000000003387819,0.000000000270595,0.000000000029686,0.000000002399442,0.000000000284141,0.009103187477939,0.000000003514694,0.000000007469123,292.296919320869904,3.904460199286052,2.618741604547198,0.879866908442036,0.000059184969656,25.924132467002011,0.000001752007220,0.000003455709463,0.000002399699455,0.000000171571263,0.000003052598490,0.000000773995442,0.000018264458215,0.000000257109315,0.000020942916107,0.000001146670526,0.000011475792062,0.000002319058983
441,Baldwin,1003,-0.831200521861337,-0.043009072938372,288.751637887770869,293.327123131983001,0.008036979921704,0.013702649779558,0.422456727373741,101806.365220763109392,0.082489200604332,0.000000009848272,0.000000013640069,0.000000019101895,0.014547872996398,0.088536637341400,170.454787516937728,101599.131343106229906,0.251301403997228,0.207313346455010,0.160910329059753,0.114339647931992,0.068773770649217,0.000901186582462,0.000003476538717,0.000003226206104,0.000010092937062,0.000000004932762,0.000001031252830,0.009987583737406,0.000006145737950,0.000002800054192,0.000000407580352,0.006242697194994,0.000007829209154,0.000000602268501,0.000002987563243,29.232544673405993,0.000000163326732,0.000000000701207,0.000000001143792,0.000000000406401,0.000000000559630,0.000000002282223,0.000000000925527,0.000000000209359,0.000000006212520,0.000000000186048,0.000000002734972,0.000000000000039,0.000000005453291,0.000000001316288,0.000000005425232,0.000000000281528,0.000000068710997,0.000000002457598,0.000000000225220,0.000000000086186,0.000000007136365,0.000000004176862,0.011948766406550,0.000000002897104,0.000000003828518,293.542298322795830,2.493748263318578,1.556460793604144,0.879988647029053,0.000159430466286,29.235735462021289,0.000002665969672,0.000005450492630,0.000004261063825,0.000000139635040,0.000002160967072,0.000000710419688,0.000015060725409,0.000000367102320,0.000030050887351,0.000003150989808,0.000010334578382,0.000001302044481
2221,Barbour,1005,-0.178294651236864,-0.090928609358090,283.713675716919170,292.746119524969231,0.009285416921739,0.009301919413331,0.986926074501393,101843.260076660779305,0.101235556113158,0.000000013745265,0.000000017359324,0.000000024043859,0.010392691700399,0.094447499457519,1263.685437879796154,100314.224012174265226,0.275067894945095,0.224662987378698,0.171514606120974,0.117823876826375,0.066733502138105,0.000932094992117,0.000003559329233,0.000004333776282,0.000009233723960,0.000000004862955,0.000003456134561,0.00986040

In [21]:
percentile_75 = get_data['c3h8'].quantile(0.90) ## calculate 75th percentile
print(percentile_75)
threshold=percentile_75

4.811531838331999e-10


In [22]:
## use county shapefile from 2008 as the shapefile before 2008 could not be found
SHAPE_PATH=os.path.join('..','..','..','..','Shapefiles','county_shapefiles','2008_county_shapefile','tl_2008_us_county.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,CNTYIDFP,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,geometry
0,19,175,00465276,19175,Union,Union County,06,H1,G4020,None,None,None,A,"POLYGON ((-94.47051 40.97504, -94.47050 40.975..."
1,19,177,00465277,19177,Van Buren,Van Buren County,06,H1,G4020,None,None,None,A,"POLYGON ((-91.96059 40.90070, -91.95925 40.900..."
2,20,097,00485013,20097,Kiowa,Kiowa County,06,H1,G4020,None,None,None,A,"POLYGON ((-99.01471 37.67895, -99.01473 37.678..."
3,20,109,00485019,20109,Logan,Logan County,06,H1,G4020,None,None,None,A,"POLYGON ((-100.89562 39.13329, -100.89404 39.1..."
4,20,003,00484971,20003,Anderson,Anderson County,06,H1,G4020,None,None,None,A,"POLYGON ((-95.51741 38.20619, -95.51741 38.206..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,35,015,00936829,35015,Eddy,Eddy County,06,H1,G4020,None,16100,None,A,"POLYGON ((-104.09554 32.00000, -104.09586 32.0..."
3229,40,015,01101795,40015,Caddo,Caddo County,06,H1,G4020,None,None,None,A,"POLYGON ((-98.36984 35.55133, -98.36964 35.551..."
3230,40,025,01101800,40025,Cimarron,Cimarron County,06,H1,G4020,None,None,None,A,"POLYGON ((-102.14252 36.50032, -102.14380 36.5..."
3231,40,069,01101822,40069,Johnston,Johnston County,06,H1,G4020,None,None,None,A,"POLYGON ((-96.74415 34.17223, -96.74416 34.172..."


In [23]:
geometry_column=county_gdf['geometry']
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x                  y
0    0    -94.470506000000000 40.975042999999999
     1    -94.470500999999999 40.975550999999996
     2    -94.470511999999999 40.978164000000000
     3    -94.470497999999992 40.978803999999997
     4    -94.470501999999996 40.979332999999997
...                       ...                ...
3232 3894 -84.275637000000003 39.289242000000002
     3895 -84.275549999999996 39.289235999999995
     3896 -84.275515999999996 39.289234000000000
     3897 -84.275508000000002 39.289232999999996
     3898 -84.275143000000000 39.289206999999998

[7781581 rows x 2 columns]

In [24]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.
        And then to find the fraction of time above percentile
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            pm_above_df: Dataframe consisting the values of the variables interpolated in the county.
    '''   
    
    
    longitude= lat_lon.loc[(var), 'x']  # get the longitude
    extract_val=len(longitude) ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    latitude=lat_lon.loc[(var),'y']    ## get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= propane.interp(longitude=lon_list, latitude=lat_list)
    
## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=year_avg_finer.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)
    
    initial_df=pd.concat(get_vals)
    summary = initial_df.groupby("valid_time").mean(["latitude", "longitude"])
    df=summary

    total_rows=len(df)

    c=0
    for i in range(0, total_rows):
        if df['c3h8'].values[i] > threshold:
            c=c+1

## convert number to a fraction
    frac_time=(c/total_rows)*100
    pm_above_df=pd.DataFrame({'propane above percentile':[frac_time]})


    return pm_above_df

In [25]:
len(county_gdf)

3233

In [26]:
%%time

## get the values for all counties

df_list_below=[]

for i in range(0,len(county_gdf)): ## loop for the all list of counties
    try:
        df_list_below.append(single_county(i))
    except:
        pass

CPU times: total: 25min 48s
Wall time: 26min 12s


In [27]:
concatenate_df=pd.concat(df_list_below)
final_df=concatenate_df.reset_index(drop=True)
final_df

,propane above percentile
0,0.616438356164384
1,3.013698630136986
2,0.273972602739726
3,0.273972602739726
4,3.013698630136986
...,...
3228,0.102739726027397
3229,2.739726027397260
3230,0.000000000000000
3231,5.787671232876712


In [28]:
## lets first create a fips column in the original shape file

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

county_gdf = county_gdf.drop([ 'STATEFP','COUNTYFP','COUNTYNS','CNTYIDFP','NAMELSAD','LSAD','CLASSFP','MTFCC',
                                 'CSAFP','CBSAFP','METDIVFP','FUNCSTAT'], axis=1)
county_gdf

,NAME,geometry,fips
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003
...,...,...,...
3228,Eddy,"POLYGON ((-104.09554 32.00000, -104.09586 32.0...",35015
3229,Caddo,"POLYGON ((-98.36984 35.55133, -98.36964 35.551...",40015
3230,Cimarron,"POLYGON ((-102.14252 36.50032, -102.14380 36.5...",40025
3231,Johnston,"POLYGON ((-96.74415 34.17223, -96.74416 34.172...",40069


In [29]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,propane above percentile
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175,0.616438356164384
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177,3.013698630136986
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097,0.273972602739726
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109,0.273972602739726
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003,3.013698630136986
...,...,...,...,...
3228,Eddy,"POLYGON ((-104.09554 32.00000, -104.09586 32.0...",35015,0.102739726027397
3229,Caddo,"POLYGON ((-98.36984 35.55133, -98.36964 35.551...",40015,2.739726027397260
3230,Cimarron,"POLYGON ((-102.14252 36.50032, -102.14380 36.5...",40025,0.000000000000000
3231,Johnston,"POLYGON ((-96.74415 34.17223, -96.74416 34.172...",40069,5.787671232876712


In [30]:
county_df=merged_df.dropna()
county_df.head()

,NAME,geometry,fips,propane above percentile
0,Union,"POLYGON ((-94.47051 40.97504, -94.47050 40.975...",19175,0.616438356164384
1,Van Buren,"POLYGON ((-91.96059 40.90070, -91.95925 40.900...",19177,3.013698630136986
2,Kiowa,"POLYGON ((-99.01471 37.67895, -99.01473 37.678...",20097,0.273972602739726
3,Logan,"POLYGON ((-100.89562 39.13329, -100.89404 39.1...",20109,0.273972602739726
4,Anderson,"POLYGON ((-95.51741 38.20619, -95.51741 38.206...",20003,3.013698630136986


In [31]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,propane above percentile
0,Union,19175,0.616438356164384
1,Van Buren,19177,3.013698630136986
2,Kiowa,20097,0.273972602739726
3,Logan,20109,0.273972602739726
4,Anderson,20003,3.013698630136986
...,...,...,...
3228,Eddy,35015,0.102739726027397
3229,Caddo,40015,2.739726027397260
3230,Cimarron,40025,0.000000000000000
3231,Johnston,40069,5.787671232876712


In [32]:
county_var.to_pickle('propane_above_percentile.pkl')